# Troubleshooting the $\Omega_k$ derivative

The computer seems to get stuck in a loop when creating omk derivative.
When I run it, it gets stuck with this as the last output:

calculating MatterPower and Window objects for  omk  derivative . . . 

neutrino_hierarchy =  normal

zs.size:  299998


In [16]:
# copy some code from FisherCl for setup in getting to that point
# preliminaries

# init parameter list:
nz=10000;lmin=2;lmax=3000;zmin=0.0;zmax=16.0;dndzMode=2
nBins=10;z0=0.5;doNorm=True;useWk=False;binSmooth=0;BPZ=True 
usePrimaryCMB=False;biasByBin=True;AccuracyBoost=3
nonlinear=False;useNoise=True;lminP=2;lmaxP=5000

# parameters passed in on earlier run:
#z0 = 0.3
z0 = 0.5 # for modelDNDZ3; should be turned on in crosspower.getDNDZinterp
#zmax = 4
zmax = 7
#nBins = 1
nBins = 6 # for beesBins; should be turned on in crosspower.tophat
lmax = 3000
lmaxP = 5000
AccuracyBoost = 3
#doNorm = True
nonlinear = True
#nonlinear = False
biasByBin = False
#hier = 'degenerate'
#hier = 'normal'
#nz = 10000
nz = 300000
usePrimaryCMB = True
#usePrimaryCMB = False
useNoise = True

# set cosmological parameters
cosParams = {
    'H0'    : None, #67.51, #setting H0=None allows cosmomc_theta to be used instead
    'cosmomc_theta'           : 1.04087e-2,
    'ombh2' : 0.02226,
    'omch2' : 0.1193,
    'omk'   : 0,
    'tau'   : 0.063,

    'As'    : 2.130e-9,
    'ns'    : 0.9653,
    'r'     : 0,
    'kPivot': 0.05,

    'w'     : -1.0, # actually is w0 but CAMB calls it w
    'wa'    : 0.0,

    # if fiducial mnu is changed, need to adjust omch2 as well
    'mnu'   : 0.06, # (eV)
    #'mnu'   : 0.058, # Lloyd suggested this value for fiducial; adjust omch2 if I do use it
    'nnu'   : 3.046,
    'standard_neutrino_neff'  : 3.046,
    'num_massive_neutrinos'   : 1,
    'neutrino_hierarchy'      : 'normal'}

if binSmooth == 0 and dndzMode == 2:
  tophatBins = True # true if bins do not overlap, false if they do
else:
  tophatBins = False
    
# observables lists
# self.obsList, obsListP created along with self.covar, covarP
nMaps = nBins+1 # +1 for kappa map
nCls = nMaps*(nMaps+1)/2 # This way removes redundancies, eg C_l^kg = C_l^gk
fieldNames = ['k']
for binNum in range(1,nBins+1):
    fieldNames.append('g'+str(binNum))
obsNames = []
for map1 in range(nMaps):
  for map2 in range(map1, nMaps):
    obsNames.append(fieldNames[map1]+','+fieldNames[map2])
if usePrimaryCMB:
    nMapsP = 2 # T,E
    #nMapsP = 3 # T,E,B
    nClsP = nMapsP*(nMapsP+1)/2
    if nMapsP == 2:
        fieldNamesP = ['T','E']
        obsNamesP = ['TT','TE','EE']
    elif nMapsP == 3:
        fieldNamesP = ['T','E','B']
        obsNamesP = ['TT','TE','EE','BB'] # TB etc?
    else:
        print 'uh oh, you didn\'t do that right.'
        
        
# modified for just one param: omk
nCosParams = 1 # omk
paramList = ['omk']
deltaP =    [ 0.01]
#deltaP =    [ 0.0]


for bin in range(nBins):
    paramList.append('bin'+str(bin+1))
nParams   = nCosParams+nBins
nCosParams = nCosParams
paramList = paramList        

In [17]:
# get MatterPower object
import crosspower as cp
reload(cp)

print 'creating MatterPower object . . . '
myPk = cp.MatterPower(nz=nz,AccuracyBoost=AccuracyBoost,nonlinear=nonlinear,
                      **cosParams)
PK,chistar,chis,dchis,zs,dzs,pars = myPk.getPKinterp()
H0 = pars.H0
Hs = myPk.Hs
zs = myPk.zs

# get Window object
print 'creating Window object . . . '
myWin = cp.Window(myPk,zmin=zmin,zmax=zmax,nBins=nBins,biasK=cp.ones,
                  biasG=cp.byeBias,dndzMode=dndzMode,z0=z0,
                  doNorm=doNorm,useWk=useWk,BPZ=BPZ,binSmooth=binSmooth,
                  biasByBin=biasByBin)

print 'done.'

creating MatterPower object . . . 
neutrino_hierarchy =  normal
starting makePkInterp.
cos_kwargs:  {}
pars:  WantCls = 1
WantTransfer = 0
WantScalars = 1
WantTensors = 0
WantVectors = 0
DoLensing = 1
want_zstar = 0
want_zdrag = 0
PK_WantTransfer = 0
NonLinear = 2
Want_CMB = 1
max_l = 3150
max_l_tensor = 600
max_eta_k = 90000.0
max_eta_k_tensor = 1200.0
omegab = 0.0488668638186
omegac = 0.261896534302
omegav = 0.687820332396
omegan = 0.00141626948299
H0 = 67.4924737242
TCMB = 2.7255
YHe = 0.245340765514
num_nu_massless = 2.03066666667
num_nu_massive = 1
nu_mass_eigenstates = 1
share_delta_neff = 0
nu_mass_degeneracies = [1.0153333333333332, 0.0, 0.0, 0.0, 0.0]
nu_mass_fractions = [1.0, 0.0, 0.0, 0.0, 0.0]
nu_mass_numbers = [1, 0, 0, 0, 0]
scalar_initial_condition = 1
OutputNormalization = 1
AccuratePolarization = 1
AccurateBB = 0
AccurateReionization = 1
MassiveNuMethod = 3
InitPower:
  tensor_parameterization = 2
  nn = 1
  an = [0.9653, 1.0, 1.0, 1.0, 1.0]
  n_run = [0.0, 0.0, 0.0, 0

In [18]:
#omegab = 0.0416887428311
#omegac = 0.223426191363
#omegav = 0.723676834099
#omegan = 0.00120823170635
#H0 = 73.0723678698

In [19]:
# get modified parameter lists for numeric differentiation
print 'creating modified parameter lists . . . '
myParams = cosParams
myParamsUpper = []
myParamsLower = []
for cParamNum in range(nCosParams):
  # add parameter dictionary to lists; HAVE TO BE COPIES!!!
  myParamsUpper.append(myParams.copy())
  myParamsLower.append(myParams.copy())
  # modify parameter number cParamNum in ditionaries
  myParamsUpper[cParamNum][paramList[cParamNum]] += deltaP[cParamNum]
  myParamsLower[cParamNum][paramList[cParamNum]] -= deltaP[cParamNum]
print 'done.'

creating modified parameter lists . . . 
done.


In [20]:
# check these
print 'myParams:'
print myParams
print 'myParamsUpper:'
print myParamsUpper
print 'myParamsLower:'
print myParamsLower

myParams:
{'cosmomc_theta': 0.0104087, 'num_massive_neutrinos': 1, 'wa': 0.0, 'tau': 0.063, 'omk': 0, 'H0': None, 'kPivot': 0.05, 'mnu': 0.06, 'standard_neutrino_neff': 3.046, 'neutrino_hierarchy': 'normal', 'As': 2.13e-09, 'r': 0, 'omch2': 0.1193, 'w': -1.0, 'nnu': 3.046, 'ns': 0.9653, 'ombh2': 0.02226}
myParamsUpper:
[{'tau': 0.063, 'wa': 0.0, 'omk': 0.01, 'mnu': 0.06, 'standard_neutrino_neff': 3.046, 'omch2': 0.1193, 'num_massive_neutrinos': 1, 'nnu': 3.046, 'cosmomc_theta': 0.0104087, 'kPivot': 0.05, 'H0': None, 'neutrino_hierarchy': 'normal', 'As': 2.13e-09, 'r': 0, 'w': -1.0, 'ns': 0.9653, 'ombh2': 0.02226}]
myParamsLower:
[{'tau': 0.063, 'wa': 0.0, 'omk': -0.01, 'mnu': 0.06, 'standard_neutrino_neff': 3.046, 'omch2': 0.1193, 'num_massive_neutrinos': 1, 'nnu': 3.046, 'cosmomc_theta': 0.0104087, 'kPivot': 0.05, 'H0': None, 'neutrino_hierarchy': 'normal', 'As': 2.13e-09, 'r': 0, 'w': -1.0, 'ns': 0.9653, 'ombh2': 0.02226}]


In [23]:
pars = myPk.pars
pars.get_omega_k()

0.0

In [9]:
parsUpper = cp.MatterPower.getPars(myPk,**myParamsUpper[0])
print 'done.'

done.


In [11]:
print parsUpper

WantCls = 1
WantTransfer = 0
WantScalars = 1
WantTensors = 0
WantVectors = 0
DoLensing = 1
want_zstar = 0
want_zdrag = 0
PK_WantTransfer = 0
NonLinear = 2
Want_CMB = 1
max_l = 3150
max_l_tensor = 600
max_eta_k = 90000.0
max_eta_k_tensor = 1200.0
omegab = 0.0416887428311
omegac = 0.223426191363
omegav = 0.723676834099
omegan = 0.00120823170635
H0 = 73.0723678698
TCMB = 2.7255
YHe = 0.245340765514
num_nu_massless = 2.03066666667
num_nu_massive = 1
nu_mass_eigenstates = 1
share_delta_neff = 0
nu_mass_degeneracies = [1.0153333333333332, 0.0, 0.0, 0.0, 0.0]
nu_mass_fractions = [1.0, 0.0, 0.0, 0.0, 0.0]
nu_mass_numbers = [1, 0, 0, 0, 0]
scalar_initial_condition = 1
OutputNormalization = 1
AccuratePolarization = 1
AccurateBB = 0
AccurateReionization = 1
MassiveNuMethod = 3
InitPower:
  tensor_parameterization = 2
  nn = 1
  an = [0.9653, 1.0, 1.0, 1.0, 1.0]
  n_run = [0.0, 0.0, 0.0, 0.0, 0.0]
  n_runrun = [0.0, 0.0, 0.0, 0.0, 0.0]
  ant = [0.0, 0.0, 0.0, 0.0, 0.0]
  nt_run = [-0.0, 0.0, 0.0, 

In [14]:
# some from makePkInterp
from camb import model
nz = 300000

print 'get_background...'
results= camb.get_background(parsUpper)
chistar = results.conformal_time(0)- model.tau_maxvis.value
chis = np.linspace(0,chistar,nz)
zs=results.redshift_at_comoving_radial_distance(chis)
zstar = zs[-1]
#Calculate array of delta_chi, and drop first and last points where things go singular
dchis = (chis[2:]-chis[:-2])/2 #overkill since chis are evenly spaced
dzs   = (  zs[2:]-  zs[:-2])/2 #not as nice as with chi since zs not evenly spaced
chis = chis[1:-1]
zs = zs[1:-1]
print 'zs.size: ',zs.size

#Get the matter power spectrum interpolation object (based on RectBivariateSpline). 
#return_z_k = True
return_z_k = False

print 'done.'

get_background...
zs.size:  299998
done.


In [15]:
# this is where it will get stuck
PK = camb.get_matter_power_interpolator(parsUpper)
print 'done'

done


In [16]:
# or not... but it did take several hours to complete.

In [17]:
# the rest of makePkInterp:
#Get H(z) values (in Mpc^-1 units)
#print 'calculating H(z) at each z...'
Hs = np.empty(nz-2)
H0 = results.h_of_z(0)
for zIndex, z in enumerate(zs):
  Hs[zIndex] = results.h_of_z(z)
Hstar = results.h_of_z(zstar)
cosmomc_theta = results.cosmomc_theta()
print 'finishing makePkInterp.'


finishing makePkInterp.


In [ ]:
# this function copied from FisherMatrix.getCrossCls; with some modifications for no longer being an object method

def getCrossCls(paramList,myParams,myPk,lmin,lmax,nBins,nz,zmin,zmax,
                dndzMode,z0,doNorm,useWk,BPZ,binSmooth,biasByBin,
                tophatBins=True,AccuracyBoost=3,nonlinear=nonlinear):
    """
        Purpose:
            get the crossCls for kappa, galaxies
        Inputs:
            paramList: like FisherMatrix.paramList, but only contains parameter
                names for those which are to be varied
                Note: a list of length 1 will cause a differently shaped array 
                to be returned
            myParams: a list of lists of parameters like FisherMatrix.cosParams
                This list must have the same length as paramList
            myPk: a MatterPower object

            nBins: number of redshift bins being used (should be nMaps-1)
            nz:
            zmin,zmax:
            dndzMode,z0,doNorm,useWk,BPZ,binSmooth,biasByBin:
            lmin,lmax: min,max ell value to use
            tophatBins: set to True to use tophat-shaped, non-overlapping bins
            AccuracyBoost:  
            nonlinear:
        Returns:
            crossCls: numpy array of shape (nMaps,nMaps,nCosParams,lmax)
              unless the lenth of paramList is 1, then shape will be 
              (nMaps,nMaps,lmax)
    """
    nCosParams = paramList.__len__()
    nMaps = nBins+1

    crossCls = np.zeros((nMaps,nMaps,nCosParams,lmax-lmin+1))
    for cParamNum in range(nCosParams):
        print 'calculating MatterPower and Window objects for ',\
              paramList[cParamNum], ' derivative . . . '

        myPks = cp.MatterPower(nz=nz,AccuracyBoost=AccuracyBoost,
                               nonlinear=nonlinear,**myParams[cParamNum])
        print 'done with MatterPower. starting Window...' 
        myWins = cp.Window(myPks,zmin=zmin,zmax=zmax,
                           nBins=nBins,biasK=cp.ones,biasG=cp.byeBias,
                           dndzMode=dndzMode,z0=z0,doNorm=doNorm,
                           useWk=useWk,BPZ=BPZ,
                           binSmooth=binSmooth,biasByBin=biasByBin)
        print 'done with Window.'

        for map1 in range(nMaps):
          if map1==0:
            cor1 = cp.Window.kappa
          else:
            cor1 = cp.Window.galaxies
          for map2 in range(map1,nMaps):
            print '  starting angular cross power spectrum ',map1,', ',map2,'... '
            if map2==0:
              cor2 = cp.Window.kappa
            else:
              cor2 = cp.Window.galaxies
            # since nonoverlapping bins have zero correlation use this condition:
            if map1==0 or map1==map2 or not tophatBins:
              ells,Cls = cp.getCl(myPks,myWins,binNum1=map1,binNum2=map2,
                                  cor1=cor1,cor2=cor2,lmin=lmin,lmax=lmax)
              crossCls[map1,map2,cParamNum] = Cls
              crossCls[map2,map1,cParamNum] = Cls #symmetric

        # ditch the MatterPower and Window objects - wait, isn't this automatic?
        del myPks
        del myWins

    # reshape for unperterbed version
    if nCosParams == 1:
        crossCls = np.reshape(crossCls,(nMaps,nMaps,lmax-lmin+1))
    return crossCls



In [ ]:
# get all cross power spectra

# If I use AOfZ not all 1, this needs to be changed 
#   to include summation over bins for kk
# If tophatBins, only the diagonal and 0th row and column will be filled

print 'starting kappa, galaxy cross power (with entire kappa) . . . '

#self.crossCls      = np.zeros((nMaps,nMaps,           lmax-lmin+1))
#self.crossClsPlus  = np.zeros((nMaps,nMaps,nCosParams,lmax-lmin+1))
#self.crossClsMinus = np.zeros((nMaps,nMaps,nCosParams,lmax-lmin+1))

# get main set of power spectra    
#crossCls = getCrossCls(['(All Fiducial)'],[myParams],myPk,
#                                 lmin,lmax,nBins,nz,zmin,zmax,dndzMode,z0,doNorm,useWk,BPZ,
#                                 binSmooth,biasByBin,tophatBins=tophatBins,
#                                 AccuracyBoost=AccuracyBoost,nonlinear=nonlinear)
# get the perturbed versions
print 'starting upper spectra for numeric derivatives . . . '
crossClsPlus = getCrossCls(paramList[:nCosParams],myParamsUpper,
                                     myPk,
                                     lmin,lmax,nBins,nz,zmin,zmax,dndzMode,z0,doNorm,useWk,BPZ,
                                     binSmooth,biasByBin,tophatBins=tophatBins,
                                     AccuracyBoost=AccuracyBoost,nonlinear=nonlinear)    
print 'starting lower spectra for numeric derivatives . . . '
crossClsMinus = getCrossCls(paramList[:nCosParams],myParamsLower,
                                      myPk,
                                      lmin,lmax,nBins,nz,zmin,zmax,dndzMode,z0,doNorm,useWk,BPZ,
                                      binSmooth,biasByBin,tophatBins=tophatBins,
                                      AccuracyBoost=AccuracyBoost,nonlinear=nonlinear)

# reset global (dark energy and AccuracyBoost) settings
lpa = 5.0 # set_for_lmax also sets lensing 
pars = myPk.getPars(lmax=lmax,lpa=lpa,AccuracyBoost=AccuracyBoost)

ells = np.arange(lmin,lmax+1)


starting kappa, galaxy cross power (with entire kappa) . . . 
starting upper spectra for numeric derivatives . . . 
calculating MatterPower and Window objects for  omk  derivative . . . 
neutrino_hierarchy =  normal
starting makePkInterp.
cos_kwargs:  {}
pars:  WantCls = 1
WantTransfer = 0
WantScalars = 1
WantTensors = 0
WantVectors = 0
DoLensing = 1
want_zstar = 0
want_zdrag = 0
PK_WantTransfer = 0
NonLinear = 2
Want_CMB = 1
max_l = 3150
max_l_tensor = 600
max_eta_k = 90000.0
max_eta_k_tensor = 1200.0
omegab = 0.0416887428311
omegac = 0.223426191363
omegav = 0.723676834099
omegan = 0.00120823170635
H0 = 73.0723678698
TCMB = 2.7255
YHe = 0.245340765514
num_nu_massless = 2.03066666667
num_nu_massive = 1
nu_mass_eigenstates = 1
share_delta_neff = 0
nu_mass_degeneracies = [1.0153333333333332, 0.0, 0.0, 0.0, 0.0]
nu_mass_fractions = [1.0, 0.0, 0.0, 0.0, 0.0]
nu_mass_numbers = [1, 0, 0, 0, 0]
scalar_initial_condition = 1
OutputNormalization = 1
AccuratePolarization = 1
AccurateBB = 0
Accura